# 🧪 Laboratorio 5: Criptografía Asimétrica — RSA

## 📚 Contenido
1. Algoritmo de Euclides y Euclides Extendido
2. Exponenciación Modular Rápida (Square-and-Multiply)
3. Generación de Claves RSA
4. Cifrado y Descifrado RSA
5. Aplicación RSA con Alfabeto A-Z
6. Casos de Prueba y Validación

---

## 🎯 Objetivos

- Implementar el **Algoritmo de Euclides** y su versión extendida
- Calcular **inversos modulares** de forma eficiente
- Implementar **exponenciación modular rápida** (crítica para RSA)
- Construir un **sistema RSA completo** de extremo a extremo
- Aplicar RSA para cifrar **mensajes de texto** usando bloques

---

## 1️⃣ Algoritmo de Euclides y Euclides Extendido

### 📖 Teoría

El **Algoritmo de Euclides** calcula el Máximo Común Divisor (MCD) de dos números:

$$\gcd(a, b) = \gcd(b, a \mod b)$$

El **Algoritmo de Euclides Extendido** encuentra coeficientes $x, y$ tales que:

$$a \cdot x + b \cdot y = \gcd(a, b)$$

Esta identidad (de Bézout) es fundamental para calcular inversos modulares.

In [ ]:
def gcd(a: int, b: int) -> int:
    """
    Calcula el Máximo Común Divisor usando el Algoritmo de Euclides.
    
    Args:
        a, b: Números enteros
    
    Returns:
        MCD(a, b)
    
    Ejemplo:
        >>> gcd(48, 18)
        6
    """
    while b != 0:
        a, b = b, a % b
    return abs(a)

# Pruebas
print("=== Algoritmo de Euclides ===")
print(f"gcd(48, 18) = {gcd(48, 18)}")
print(f"gcd(17, 3120) = {gcd(17, 3120)}")
print(f"gcd(100, 35) = {gcd(100, 35)}")

In [ ]:
def extended_gcd(a: int, b: int) -> tuple[int, int, int]:
    """
    Algoritmo de Euclides Extendido.
    Retorna (gcd, x, y) tal que a*x + b*y = gcd(a, b)
    
    Args:
        a, b: Números enteros
    
    Returns:
        Tupla (gcd, x, y)
    
    Ejemplo:
        >>> extended_gcd(17, 3120)
        (1, -367, 2)
    """
    if b == 0:
        return abs(a), 1 if a >= 0 else -1, 0
    
    # Recursión
    gcd_val, x1, y1 = extended_gcd(b, a % b)
    
    # Actualizar coeficientes usando las relaciones de Bézout
    x = y1
    y = x1 - (a // b) * y1
    
    return gcd_val, x, y

# Pruebas
print("\n=== Algoritmo de Euclides Extendido ===")
g, x, y = extended_gcd(17, 3120)
print(f"extended_gcd(17, 3120) = ({g}, {x}, {y})")
print(f"Verificación: 17*{x} + 3120*{y} = {17*x + 3120*y}")

g, x, y = extended_gcd(48, 18)
print(f"\nextended_gcd(48, 18) = ({g}, {x}, {y})")
print(f"Verificación: 48*{x} + 18*{y} = {48*x + 18*y}")

In [ ]:
def modular_inverse(a: int, m: int) -> int:
    """
    Calcula el inverso modular de 'a' módulo 'm'.
    Encuentra x tal que (a * x) % m = 1
    
    Args:
        a: Número a invertir
        m: Módulo
    
    Returns:
        Inverso modular de a módulo m
    
    Raises:
        ValueError: Si no existe inverso (gcd(a,m) ≠ 1)
    
    Ejemplo:
        >>> modular_inverse(17, 3120)
        2753
    """
    gcd_val, x, _ = extended_gcd(a, m)
    
    if gcd_val != 1:
        raise ValueError(f"No existe inverso: gcd({a}, {m}) = {gcd_val} ≠ 1")
    
    # Asegurar que el resultado sea positivo
    return x % m

# Pruebas
print("\n=== Inverso Modular ===")
inv = modular_inverse(17, 3120)
print(f"modular_inverse(17, 3120) = {inv}")
print(f"Verificación: (17 * {inv}) % 3120 = {(17 * inv) % 3120}")

inv2 = modular_inverse(7, 26)
print(f"\nmodular_inverse(7, 26) = {inv2}")
print(f"Verificación: (7 * {inv2}) % 26 = {(7 * inv2) % 26}")

---

## 2️⃣ Exponenciación Modular Rápida

### 📖 Teoría

El algoritmo **Square-and-Multiply** calcula $b^e \mod m$ eficientemente:

1. Convertir $e$ a binario
2. Procesar cada bit de izquierda a derecha:
   - **Siempre**: Elevar al cuadrado el resultado
   - **Si bit = 1**: Multiplicar por la base

**Complejidad**: $O(\log e)$ multiplicaciones en lugar de $O(e)$

**Ejemplo**: $5^{13} \mod 7$
- $13_{10} = 1101_2$
- Bits: 1, 1, 0, 1

In [ ]:
def modular_exponentiation(base: int, exp: int, mod: int) -> int:
    """
    Exponenciación modular rápida: base^exp mod mod
    Algoritmo: Square-and-Multiply
    
    Args:
        base: Base
        exp: Exponente (debe ser >= 0)
        mod: Módulo (debe ser > 0)
    
    Returns:
        base^exp mod mod
    
    Ejemplo:
        >>> modular_exponentiation(5, 13, 7)
        3
    """
    if mod == 1:
        return 0
    
    result = 1
    base = base % mod
    
    while exp > 0:
        # Si el bit actual es 1, multiplicar por base
        if exp & 1:
            result = (result * base) % mod
        
        # Elevar base al cuadrado
        base = (base * base) % mod
        
        # Desplazar exponente (dividir entre 2)
        exp >>= 1
    
    return result

# Pruebas
print("=== Exponenciación Modular Rápida ===")
print(f"5^13 mod 7 = {modular_exponentiation(5, 13, 7)}")
print(f"65^17 mod 3233 = {modular_exponentiation(65, 17, 3233)}")
print(f"2^10 mod 1000 = {modular_exponentiation(2, 10, 1000)}")

# Comparar con pow() de Python
assert modular_exponentiation(5, 13, 7) == pow(5, 13, 7)
assert modular_exponentiation(65, 17, 3233) == pow(65, 17, 3233)
print("\n✓ Todas las verificaciones con pow() pasaron exitosamente")

### 🔍 Visualización del Algoritmo

In [ ]:
def modular_exponentiation_traced(base: int, exp: int, mod: int) -> int:
    """
    Versión con trazabilidad para propósitos educativos.
    Muestra cada paso del algoritmo.
    """
    print(f"\nCalculando {base}^{exp} mod {mod}")
    print("="*70)
    
    # Mostrar exponente en binario
    exp_binary = bin(exp)[2:]
    print(f"Exponente: {exp} = {exp_binary} (binario)")
    print(f"\n{'Paso':<6} {'Bit':<6} {'Operación':<40} {'Resultado':<10}")
    print("-"*70)
    
    result = 1
    base = base % mod
    temp_exp = exp
    step = 0
    
    # Obtener todos los bits
    bits = []
    while temp_exp > 0:
        bits.append(temp_exp & 1)
        temp_exp >>= 1
    bits.reverse()
    
    # Procesar primer bit (siempre 1)
    result = base
    print(f"{step+1:<6} {1:<6} result = base = {base}{' '*26}{result:<10}")
    step += 1
    
    # Procesar resto de bits
    for bit in bits[1:]:
        # Siempre elevar al cuadrado
        old_result = result
        result = (result * result) % mod
        print(f"{step+1:<6} {'-':<6} result = {old_result}² mod {mod}{' '*16}{result:<10}")
        step += 1
        
        # Si bit es 1, multiplicar por base
        if bit:
            old_result = result
            result = (result * base) % mod
            print(f"{step+1:<6} {1:<6} result = {old_result} × {base} mod {mod}{' '*14}{result:<10}")
            step += 1
    
    print("-"*70)
    print(f"\n✓ Resultado: {base}^{exp} mod {mod} = {result}\n")
    return result

# Ejemplo con trazabilidad
result = modular_exponentiation_traced(5, 13, 7)

---

## 3️⃣ Generación de Claves RSA

### 📖 Teoría

**Pasos para generar claves RSA**:

1. Elegir dos primos distintos $p$ y $q$
2. Calcular $n = p \times q$ (módulo público)
3. Calcular $\phi(n) = (p-1)(q-1)$ (función de Euler)
4. Elegir $e$ tal que $1 < e < \phi(n)$ y $\gcd(e, \phi(n)) = 1$
5. Calcular $d = e^{-1} \mod \phi(n)$ (inverso modular)

**Claves**:
- **Pública**: $(e, n)$
- **Privada**: $(d, n)$

In [ ]:
def generate_phi(p: int, q: int) -> int:
    """
    Calcula φ(n) = (p-1)(q-1) donde n = p*q
    
    Args:
        p, q: Números primos
    
    Returns:
        φ(n)
    """
    return (p - 1) * (q - 1)

# Prueba
print("=== Función φ (Euler) ===")
p, q = 61, 53
phi = generate_phi(p, q)
print(f"p = {p}, q = {q}")
print(f"φ({p}×{q}) = ({p}-1)×({q}-1) = {phi}")

In [ ]:
def generate_keys(p: int, q: int, e: int) -> tuple[tuple[int, int], tuple[int, int]]:
    """
    Genera claves pública y privada RSA.
    
    Args:
        p, q: Números primos distintos
        e: Exponente público (debe cumplir gcd(e, φ(n)) = 1)
    
    Returns:
        Tupla ((e, n), (d, n)) con clave pública y privada
    
    Raises:
        ValueError: Si las condiciones RSA no se cumplen
    """
    # Validaciones
    if p == q:
        raise ValueError("p y q deben ser distintos")
    
    # Calcular n y φ(n)
    n = p * q
    phi = generate_phi(p, q)
    
    # Verificar que e sea válido
    if not (1 < e < phi):
        raise ValueError(f"e debe estar en el rango (1, {phi})")
    
    if gcd(e, phi) != 1:
        raise ValueError(f"gcd(e, φ(n)) = gcd({e}, {phi}) ≠ 1")
    
    # Calcular d (inverso modular de e)
    d = modular_inverse(e, phi)
    
    # Verificar que (e * d) mod φ(n) = 1
    assert (e * d) % phi == 1, "Error en cálculo de d"
    
    public_key = (e, n)
    private_key = (d, n)
    
    return public_key, private_key

# Prueba con el ejemplo clásico
print("\n=== Generación de Claves RSA ===")
p, q, e = 61, 53, 17

print(f"Parámetros:")
print(f"  p = {p} (primo)")
print(f"  q = {q} (primo)")
print(f"  e = {e} (exponente público)")

public_key, private_key = generate_keys(p, q, e)

print(f"\nClaves generadas:")
print(f"  Clave pública:  (e={public_key[0]}, n={public_key[1]})")
print(f"  Clave privada:  (d={private_key[0]}, n={private_key[1]})")

# Verificación
n = public_key[1]
d = private_key[0]
phi = generate_phi(p, q)
print(f"\nVerificaciones:")
print(f"  n = p×q = {p}×{q} = {n} ✓")
print(f"  φ(n) = {phi} ✓")
print(f"  (e×d) mod φ(n) = ({e}×{d}) mod {phi} = {(e*d) % phi} ✓")

---

## 4️⃣ Cifrado y Descifrado RSA

### 📖 Teoría

**Cifrado**: $C = M^e \mod n$

**Descifrado**: $M = C^d \mod n$

**Teorema fundamental**: $(M^e)^d \equiv M \pmod{n}$ (por el Teorema de Euler)

In [ ]:
def rsa_encrypt(message: int, e: int, n: int) -> int:
    """
    Cifra un mensaje usando RSA.
    
    Args:
        message: Mensaje en claro (entero < n)
        e: Exponente público
        n: Módulo público
    
    Returns:
        Mensaje cifrado C = M^e mod n
    """
    if message >= n:
        raise ValueError(f"Mensaje {message} debe ser < n ({n})")
    
    return modular_exponentiation(message, e, n)


def rsa_decrypt(cipher: int, d: int, n: int) -> int:
    """
    Descifra un mensaje usando RSA.
    
    Args:
        cipher: Mensaje cifrado
        d: Exponente privado
        n: Módulo público
    
    Returns:
        Mensaje original M = C^d mod n
    """
    return modular_exponentiation(cipher, d, n)

# Pruebas
print("=== Cifrado y Descifrado RSA ===")

# Usar claves generadas anteriormente
e, n = public_key
d, _ = private_key

# Cifrar letra 'A' (código 65)
M = 65
print(f"\nMensaje original: M = {M} (letra 'A')")

C = rsa_encrypt(M, e, n)
print(f"Mensaje cifrado:  C = {M}^{e} mod {n} = {C}")

M2 = rsa_decrypt(C, d, n)
print(f"Mensaje descifrado: M = {C}^{d} mod {n} = {M2}")

print(f"\n✓ Verificación: M == M2 ? {M == M2}")

# Más ejemplos
print("\n" + "="*70)
test_messages = [1, 100, 500, 1000, 3232]
print(f"\n{'M':<10} {'C':<10} {'M recuperado':<15} {'Correcto?':<10}")
print("-"*50)
for m in test_messages:
    c = rsa_encrypt(m, e, n)
    m_recovered = rsa_decrypt(c, d, n)
    print(f"{m:<10} {c:<10} {m_recovered:<15} {'✓' if m == m_recovered else '✗':<10}")

---

## 5️⃣ Aplicación RSA con Alfabeto A-Z

### 📖 Teoría

Para cifrar texto:
1. **Mapear**: A=00, B=01, ..., Z=25 (pares de dígitos)
2. **Agrupar**: Formar bloques < n
3. **Cifrar**: Cada bloque independientemente
4. **Guardar**: Número de letras por bloque (para reconstrucción)

In [ ]:
def text_to_numbers(text: str) -> str:
    """
    Convierte texto a representación numérica.
    A=00, B=01, ..., Z=25
    
    Args:
        text: Texto en mayúsculas (A-Z)
    
    Returns:
        String numérico con pares de dígitos
    """
    text = text.upper()
    numeric_str = ""
    
    for char in text:
        if 'A' <= char <= 'Z':
            value = ord(char) - ord('A')
            numeric_str += f"{value:02d}"
    
    return numeric_str


def numbers_to_text(numeric_str: str) -> str:
    """
    Convierte representación numérica de vuelta a texto.
    
    Args:
        numeric_str: String de dígitos (pares)
    
    Returns:
        Texto reconstruido
    """
    text = ""
    
    for i in range(0, len(numeric_str), 2):
        if i + 1 < len(numeric_str):
            pair = numeric_str[i:i+2]
            value = int(pair)
            if 0 <= value <= 25:
                text += chr(ord('A') + value)
    
    return text

# Pruebas
print("=== Conversión Texto ↔ Números ===")
test_text = "HELLO"
numeric = text_to_numbers(test_text)
recovered = numbers_to_text(numeric)

print(f"Texto original:   {test_text}")
print(f"Numérico:         {numeric}")
print(f"Texto recuperado: {recovered}")
print(f"\n✓ Correcto: {test_text == recovered}")

In [ ]:
def split_into_blocks(numeric_str: str, n: int) -> list[tuple[int, int]]:
    """
    Divide la cadena numérica en bloques < n.
    Usa algoritmo greedy para maximizar tamaño de bloques.
    
    Args:
        numeric_str: Cadena numérica
        n: Módulo RSA
    
    Returns:
        Lista de tuplas (valor_bloque, num_letras)
    """
    blocks = []
    i = 0
    
    while i < len(numeric_str):
        current_block = ""
        letters_count = 0
        
        while i < len(numeric_str):
            if i + 1 < len(numeric_str):
                next_pair = numeric_str[i:i+2]
                test_block = current_block + next_pair
                test_value = int(test_block)
                
                if test_value < n:
                    current_block = test_block
                    letters_count += 1
                    i += 2
                else:
                    break
            else:
                break
        
        if current_block:
            blocks.append((int(current_block), letters_count))
    
    return blocks

# Prueba
print("\n=== División en Bloques ===")
numeric = text_to_numbers("HELLO")
n = 3233
blocks = split_into_blocks(numeric, n)

print(f"Cadena numérica: {numeric}")
print(f"Módulo n: {n}")
print(f"\nBloques generados:")
for i, (value, num_letters) in enumerate(blocks, 1):
    print(f"  Bloque {i}: {value:4d} ({num_letters} letras) < {n}")

In [ ]:
def encrypt_message(plaintext: str, e: int, n: int) -> list[tuple[int, int]]:
    """
    Cifra un mensaje de texto completo.
    
    Args:
        plaintext: Mensaje en texto claro
        e: Exponente público
        n: Módulo público
    
    Returns:
        Lista de tuplas (bloque_cifrado, num_letras)
    """
    # Convertir a números
    numeric_str = text_to_numbers(plaintext)
    
    # Dividir en bloques
    blocks = split_into_blocks(numeric_str, n)
    
    # Cifrar cada bloque
    encrypted_blocks = []
    for block_value, num_letters in blocks:
        cipher_value = rsa_encrypt(block_value, e, n)
        encrypted_blocks.append((cipher_value, num_letters))
    
    return encrypted_blocks


def decrypt_message(encrypted_blocks: list[tuple[int, int]], d: int, n: int) -> str:
    """
    Descifra un mensaje completo.
    
    Args:
        encrypted_blocks: Lista de tuplas (bloque_cifrado, num_letras)
        d: Exponente privado
        n: Módulo público
    
    Returns:
        Mensaje en texto claro
    """
    numeric_str = ""
    
    for cipher_value, num_letters in encrypted_blocks:
        # Descifrar bloque
        block_value = rsa_decrypt(cipher_value, d, n)
        
        # Reconstruir con padding (2 dígitos por letra)
        expected_length = num_letters * 2
        block_str = str(block_value).zfill(expected_length)
        
        numeric_str += block_str
    
    # Convertir de vuelta a texto
    return numbers_to_text(numeric_str)

# Prueba completa
print("\n" + "="*70)
print("=== Cifrado/Descifrado de Mensajes Completos ===")
print("="*70)

plaintext = "HELLO"
e, n = public_key
d, _ = private_key

print(f"\nMensaje original: {plaintext}")

# Cifrar
cipher_blocks = encrypt_message(plaintext, e, n)
print(f"\nBloques cifrados:")
for i, (cipher, num_letters) in enumerate(cipher_blocks, 1):
    print(f"  Bloque {i}: {cipher} ({num_letters} letras)")

# Descifrar
decrypted = decrypt_message(cipher_blocks, d, n)
print(f"\nMensaje descifrado: {decrypted}")

print(f"\n✓ Verificación: {plaintext} == {decrypted} ? {plaintext == decrypted}")

---

## 6️⃣ Casos de Prueba y Validación

### Pruebas Exhaustivas

In [ ]:
print("="*70)
print("SUITE DE PRUEBAS COMPLETA")
print("="*70)

# Configuración de prueba
p, q, e = 61, 53, 17
public_key, private_key = generate_keys(p, q, e)
e, n = public_key
d, _ = private_key

# Lista de mensajes de prueba
test_messages = [
    "A",
    "HELLO",
    "ATTACK",
    "CRYPTOGRAPHY",
    "THEQUICKBROWNFOX",
    "Z" * 10  # Repetición
]

print(f"\nClaves: e={e}, d={d}, n={n}")
print(f"\n{'Mensaje':<20} {'Bloques cifrados':<30} {'Recuperado':<20} {'OK?'}")
print("-"*80)

all_passed = True

for msg in test_messages:
    try:
        # Cifrar
        encrypted = encrypt_message(msg, e, n)
        cipher_str = str([c for c, _ in encrypted])
        
        # Descifrar
        decrypted = decrypt_message(encrypted, d, n)
        
        # Verificar
        passed = (msg == decrypted)
        all_passed = all_passed and passed
        
        status = "✓" if passed else "✗"
        print(f"{msg:<20} {cipher_str:<30} {decrypted:<20} {status}")
        
    except Exception as ex:
        print(f"{msg:<20} ERROR: {str(ex)}")
        all_passed = False

print("-"*80)
print(f"\n{'✓✓✓ TODAS LAS PRUEBAS PASARON ✓✓✓' if all_passed else '✗✗✗ ALGUNAS PRUEBAS FALLARON ✗✗✗'}")

### Análisis de Seguridad Básico

In [ ]:
print("\n" + "="*70)
print("ANÁLISIS DE PARÁMETROS RSA")
print("="*70)

print(f"\n1. Tamaño de clave:")
print(f"   n = {n}")
print(f"   Bits: {n.bit_length()} bits")
print(f"   Nota: Para seguridad real, se necesitan al menos 2048 bits")

print(f"\n2. Primos utilizados:")
print(f"   p = {p}")
print(f"   q = {q}")
print(f"   p ≠ q: {p != q} ✓")

print(f"\n3. Exponente público:")
print(f"   e = {e}")
phi = (p-1) * (q-1)
print(f"   φ(n) = {phi}")
print(f"   gcd(e, φ(n)) = {gcd(e, phi)} ✓")

print(f"\n4. Exponente privado:")
print(f"   d = {d}")
print(f"   (e × d) mod φ(n) = {(e * d) % phi} ✓")

print(f"\n5. Capacidad de mensajes:")
max_letters = 0
test_block = ""
while int(test_block + "25") < n:  # 'Z' = 25
    test_block += "25"
    max_letters += 1
print(f"   Máximo de letras por bloque: {max_letters}")
print(f"   Valor máximo de bloque: {int(test_block)}")

---

## 📊 Resumen y Conclusiones

### Algoritmos Implementados

✅ **Euclides y Euclides Extendido**: Cálculo de MCD e inversos modulares

✅ **Exponenciación Modular Rápida**: Algoritmo eficiente O(log n)

✅ **Generación de Claves RSA**: Validación completa de parámetros

✅ **Cifrado/Descifrado**: Implementación correcta del esquema RSA

✅ **Manejo de Texto**: Sistema de bloques con padding correcto

### Conceptos Clave Aprendidos

1. **La seguridad de RSA** se basa en la dificultad de factorizar n = p×q
2. **El Teorema de Euler** garantiza que (M^e)^d ≡ M (mod n)
3. **La exponenciación modular** es crítica para la eficiencia
4. **El padding** es esencial para reconstruir mensajes correctamente

### Limitaciones de esta Implementación

⚠️ **Educativa**: No usar en producción
- Primos pequeños (61, 53) son trivialmente factorizables
- No hay padding criptográfico (OAEP)
- No hay protección contra ataques de temporización
- Para uso real: usar bibliotecas como `cryptography` o `PyCryptodome`

---

## 🎓 Ejercicios Adicionales

1. Implementar un test de primalidad (Miller-Rabin)
2. Generar primos aleatorios de mayor tamaño
3. Comparar tiempos de ejecución con/sin exponenciación rápida
4. Implementar padding PKCS#1 v1.5
5. Cifrar/descifrar archivos pequeños

---

## 📚 Referencias

- **Rivest, Shamir, Adleman** (1977): "A Method for Obtaining Digital Signatures and Public-Key Cryptosystems"
- **Cormen et al.**: "Introduction to Algorithms" (Capítulo de Teoría de Números)
- **RFC 8017**: PKCS #1: RSA Cryptography Specifications

---

*Laboratorio completado exitosamente* ✅